In [4]:
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from sklearn import preprocessing
from sklearn.cluster import DBSCAN
from sklearn.feature_selection import SelectKBest, chi2

In [5]:
# import datasets
orders = pd.read_csv('./data/olist_orders_dataset.csv')
customers = pd.read_csv('./data/olist_customers_dataset.csv')
products_translations = pd.read_csv('./data/product_category_name_translation.csv')
products = pd.read_csv("./data/olist_products_dataset.csv")
orderreviews_data = pd.read_csv('./data/olist_order_reviews_dataset.csv')
order_items = pd.read_csv("./data/olist_order_items_dataset.csv")
payments = pd.read_csv('./data/olist_order_payments_dataset.csv')

In [6]:
# convert the order delivered carrier date into specified format
orders['order_delivered_carrier_date'] = orders[['order_delivered_carrier_date']] \
    .apply(lambda x: pd.to_datetime(x, errors='coerce', format='%Y-%m-%d %H:%M:%S')) \
    .apply(lambda x: x.dt.date) \
    .astype('datetime64')

In [7]:
# drop nulls from order delivered carrier date
orders = orders.dropna(subset=['order_delivered_carrier_date'])

In [8]:
# We'll be taking the recent day plus 1 day as the present date for the rfm analysis
recent_date = orders['order_delivered_carrier_date'].max() + timedelta(days=1)

In [9]:
# form the rfm dataframe by grouping by customer id and using aggregations
rfm_df = merged_data.groupby('customer_id').agg(
    recency=('order_delivered_carrier_date', lambda x: (recent_date - x.max()).days),
    frequency=('payment_sequential', 'count'),
    monetary=('payment_value', 'sum'))

In [10]:
# drop outliers which are outside the interquantile range
def drop_outliers(df, column):
    quantile1 = df[column].quantile(0.05)
    quantile3 = df[column].quantile(0.95)
    iqr = quantile3 - quantile1
    high_fence = quantile3 + 1.5 * iqr
    low_fence = quantile1 - 1.5 * iqr
    outliers_index = df.loc[(df[column] < low_fence) | (df[column] > high_fence)].index
    df = pd.DataFrame(df.drop(outliers_index.to_list(), axis=0, inplace=True))
    return df

In [11]:
drop_outliers(rfm_df, 'frequency')

In [12]:
drop_outliers(rfm_df, 'monetary')

### Assign Scores for each column

In [13]:
rfm_df["recency_score"] = pd.qcut(rfm_df['recency'], 5, labels=[5, 4, 3, 2, 1])
rfm_df["frequency_score"] = pd.qcut(rfm_df['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm_df["monetary_score"] = pd.qcut(rfm_df['monetary'], 5, labels=[1, 2, 3, 4, 5])

In [14]:
# club all the recency, frequency and monetary scores together to get asingle rfm score
rfm_df['rfm_score'] = rfm_df.recency_score.astype(str) + rfm_df.frequency_score.astype(
    str) + rfm_df.monetary_score.astype(str)

In [15]:
rfm_score_mappings = {
    r'155|154|144|214|215|115|114|113': 'Cannot Lose Them',
    r'255|254|245|244|253|252|243|242|235|234|225|224|153|152|145|143|142|135|134|133|125|124': 'At Risk',
    r'331|321|312|221|213|231|241|251': 'About To Sleep',
    r'535|534|443|434|343|334|325|324': 'Need Attention',
    r'543|444|435|355|354|345|344|335': 'Loyal',
    r'555|554|544|545|454|455|445': 'Champions'
                                    r'111|112|121|131|141|151': 'Lost customers',
r'332|322|233|232|223|222|132|123|122|212|211': 'Hibernating customers',
r'525|524|523|522|521|515|514|513|425|424|413|414|415|315|314|313': 'Promising',
r'512|511|422|421|412|411|311': 'New Customers',
r'553|551|552|541|542|533|532|531|452|451|442|441|431|453|433|432|423|353|352|351|342|341|333|323': 'Potential Loyalist'
}

In [16]:
# create rfm segments by using rfm score to segments mappings
rfm_df['segment'] = rfm_df['recency_score'].astype(str) + rfm_df['frequency_score'].astype(str) + rfm_df[
    'monetary_score'].astype(str)
rfm_df['segment'] = rfm_df['segment'].replace(rfm_score_mappings, regex=True)
rfm_df.head()

In [17]:
rfm_df = rfm_df[['recency', 'frequency', 'monetary', 'rfm_score', 'segment']]

In [128]:
# joining our data into one single dataframe
df = rfm_df.merge(orders, how='inner', on=['customer_id'])
df = df.merge(customers, how="inner", on=["customer_id"])
df = df.merge(order_items, how="inner", on=["order_id"])
df = df.merge(payments, how="inner", on=["order_id"])
df = df.merge(orderreviews_data, how="inner", on=["order_id"])
df = df.merge(products, how="inner", on=["product_id"])
df = df.merge(products_translations, how="inner", on=["product_category_name"])

df.head()

,customer_id,recency,frequency,monetary,rfm_score,segment,order_id,order_status,order_purchase_timestamp,order_approved_at,...,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,00012a2ce6f8dcda20d059ce98491703,299,1,114.74,213,About To Sleep,5f79b5b0931d63f1a42989eb65b9da6e,delivered,2017-11-14 16:08:26,2017-11-14 16:35:32,...,2017-12-04 12:00:07,brinquedos,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys
1,13d6935a154e88a19cec61280ff67217,373,1,109.70,213,About To Sleep,12d9723638a09350ec19ea8d66073249,delivered,2017-09-01 20:05:31,2017-09-01 22:25:46,...,2017-09-13 02:13:46,brinquedos,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys
2,279375c3290f45664fb0b20cde471987,328,1,107.80,213,About To Sleep,cca53f6baff9286b728cc328ca7a2d37,delivered,2017-10-16 19:01:58,2017-10-16 19:40:08,...,2017-11-01 18:17:18,brinquedos,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys
3,2b685398c61b72f24fce6c289aa1d810,77,1,98.29,513,Promising,d266e87fb98640ecbbe91425c76e72f3,delivered,2018-06-26 18:03:15,2018-06-26 18:18:44,...,2018-07-24 14:20:39,brinquedos,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys
4,323a3f2d3b2d3413da554ffd9e722d5d,386,1,291.47,215,Cannot Lose Them,62f7289ebfb6bb8bcf133442c92558d4,delivered,2017-08-21 17:17:20,2017-08-21 17:31:35,...,2017-08-26 21:07:14,brinquedos,55.0,889.0,1.0,4267.0,46.0,39.0,22.0,toys


In [39]:
# label encode all the columns which are of type object
label_encoder = preprocessing.LabelEncoder()

In [41]:
train_df['order_status'] = label_encoder.fit_transform(train_df['order_status'])
train_df['customer_zip_code_prefix'] = label_encoder.fit_transform(train_df['customer_zip_code_prefix'])
train_df['customer_city'] = label_encoder.fit_transform(train_df['customer_city'])
train_df['customer_state'] = label_encoder.fit_transform(train_df['customer_state'])
train_df['payment_type'] = label_encoder.fit_transform(train_df['payment_type'])
train_df['product_category_name_english'] = label_encoder.fit_transform(train_df['product_category_name_english'])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18368\3954386138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['order_status']= label_encoder.fit_transform(train_df['order_status'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18368\3954386138.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['customer_zip_code_prefix']= label_encoder.fit_transform(train_df['customer_zip_code_prefix'])
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18368\3954386138.py:3: SettingWithCopyWarning: 
A value

### Select k best features using chi square 

In [43]:
# choose top k features using chi square test
# we are choosing only 2 features as the runtime for the algorithm increases with more number of variables
select = SelectKBest(chi2, k=2)

In [44]:
z = select.fit_transform(train_df.drop('rfm_score', axis=1), train_df['rfm_score'])

In [45]:
filter = select.get_support()

In [46]:
z = np.array(train_df.drop('rfm_score', axis=1).columns)

In [47]:
z[filter]

array(['customer_zip_code_prefix', 'product_weight_g'], dtype=object)

In [156]:
train_df = df[['customer_zip_code_prefix', 'product_weight_g', 'recency', 'frequency', 'monetary']]

### Clustering with DBSCAN

In [185]:
%%time
db = DBSCAN().fit(train_df)

CPU times: total: 516 ms
Wall time: 163 ms


In [187]:
labels = db.labels_

In [159]:
train_df['labels'] = labels

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18368\2517713918.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['labels'] = labels


### get the unsupervised performance metrics

In [3]:
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(train_df, labels))
print("Calinski Harabasz Coefficient: %0.3f" % metrics.calinski_harabasz_score(train_df, labels))

Silhouette Coefficient:  -0.817
Calinski Harabasz Coefficient:  12.036
